# **<p style="text-align: center;">Aprendizaje por refuerzo - Ingeniería en Inteligencia Artificial</p>**
## **<p style="text-align: center;">Trabajo práctico Final - SAC</p>**

#### <p style="text-align: center;">Pettinari Fausto, Schuemer Ignacio, Torres Santiago </p>
#### <p style="text-align: center;">Profesores: Claudio Pose, Gabriel Torre, Nicolás Romero, Tomás Chimenti</p>

# **Quadratic Action Reward Enviroment - *Entorno de Prueba***

In [9]:
import gymnasium as gym
from envs import QuadraticActionRewardEnv

In [10]:
gym.register(
    id="gymnasium_env/QuadraticActionRewardEnv-v0",
    entry_point=QuadraticActionRewardEnv
)
env = gym.make("gymnasium_env/QuadraticActionRewardEnv-v0")

/home/ignacio/miniconda3/lib/python3.12/site-packages/gymnasium/envs/registration.py:636: UserWarning: WARN: Overriding environment gymnasium_env/QuadraticActionRewardEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


#### **Logging & Debugging (Random Agent)**

In [11]:
from torch.utils.tensorboard import SummaryWriter
import sys
import os
sys.path.append(os.path.join('..'))
from src.random_agent import random_agent_loop

In [12]:
seed = 42
writer = SummaryWriter(f"../runs/QuadraticActionRewardEnv")
num_episodes = 5000
# random_agent_loop(env, num_episodes, writer, seed)



## **SAC Agent**

In [13]:
from src.SAC import SAC
from src.hyperparameters import SACConfig

In [14]:
from dataclasses import replace
from src.alpha_scheduler import FixedAlpha

base = SACConfig()
experiment_cfg = SACConfig(
    sac=replace(base.sac, gamma=0.99, tau=0.01, alpha=FixedAlpha(0.1)),
    q_net=replace(base.q_net, hidden_sizes=[10, 10], hidden_layers_act="relu"),
    policy_net=replace(base.policy_net, log_std_min=-10, log_std_max=1, hidden_sizes=[10,10], hidden_layers_act="relu"),
    buffer=replace(base.buffer, capacity=1000),
    train=replace(base.train, batch_size=256, device="cuda"),
    logger=replace(base.logger, env_name="QuadraticActionRewardEnv", log_dir="../runs")
)

In [15]:
SAC_agent = SAC(env=env, config=experiment_cfg)

In [16]:
SAC_agent.run_training_loop(num_episodes=5000, tqdm_disable=False, print_rewards=False)

100%|██████████| 5000/5000 [01:15<00:00, 66.24it/s] 


{'total_episodes': 5000,
 'best_avg_return': np.float32(-0.09574162),
 'final_avg_return': np.float32(-0.100555696)}